In [4]:
!pip freeze | grep scipy

scipy==1.11.4


In [5]:
!pip install bert-extractive-summarizer

In [26]:
!pip install tqdm

In [3]:
import pandas as pd
from ast import literal_eval
import nltk
import re
from gensim.models import Word2Vec
from sklearn.manifold import TSNE
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter

In [6]:
from summarizer import Summarizer

In [27]:
from tqdm import tqdm
from more_itertools import chunked

# Load Datasets

We load the preprocessed datasets. Make sure to run the previous notebooks first, so the files are present. I also shared the files on drive, so everyone can acces them.

## media

In [7]:
url = 'https://drive.google.com/file/d/1xNDT_EA_6i0lvL03Tzs2TmH6qrWbjIiT/view?usp=sharing'
path = 'https://drive.google.com/uc?export=download&confirm=1&id='+url.split('/')[-2]
df_media = pd.read_csv(path)
df_media

,title,date,author,content,domain,url,token_content
0,Qatar to Slash Emissions as LNG Expansion Adva...,2021-01-13,NaN,Qatar Petroleum ( QP) is targeting aggressive ...,energyintel,https://www.energyintel.com/0000017b-a7dc-de4c...,"['qatar', 'petroleum', 'qp', 'target', 'aggres..."
1,India Launches Its First 700 MW PHWR,2021-01-15,NaN,Nuclear Power Corp. of India Ltd. ( NPCIL) syn...,energyintel,https://www.energyintel.com/0000017b-a7dc-de4c...,"['nuclear', 'power', 'corp', 'india', 'ltd', '..."
2,New Chapter for US-China Energy Trade,2021-01-20,NaN,New US President Joe Biden took office this we...,energyintel,https://www.energyintel.com/0000017b-a7dc-de4c...,"['new', 'us', 'presid', 'joe', 'biden', 'take'..."
3,Japan: Slow Restarts Cast Doubt on 2030 Energy...,2021-01-22,NaN,The slow pace of Japanese reactor restarts con...,energyintel,https://www.energyintel.com/0000017b-a7dc-de4c...,"['slow', 'pace', 'japanes', 'reactor', 'restar..."
4,NYC Pension Funds to Divest Fossil Fuel Shares,2021-01-25,NaN,Two of New York City's largest pension funds s...,energyintel,https://www.energyintel.com/0000017b-a7dc-de4c...,"['two', 'new', 'york', 'citi', 'larg', 'pensio..."
...,...,...,...,...,...,...,...
9583,Strata Clean Energy Nets $ 300 Million in Fund...,2023-11-06,NaN,Strata Clean Energy has closed a $ 300 million...,solarindustrymag,https://solarindustrymag.com/strata-clean-ener...,"['strata', 'clean', 'energi', 'close', 'millio..."
9584,Orsted Deploying SparkCognition Renewable Suit...,2023-11-07,NaN,Global renewable energy developer Ørsted is de...,solarindustrymag,https://solarindustrymag.com/orsted-deploying-...,"['global', 'renew', 'energi', 'develop', 'ørst..."
9585,Veolia Has Plans for 5 MW of Solar in Arkansas,2023-11-07,NaN,"Veolia North America, a provider of environmen...",solarindustrymag,https://solarindustrymag.com/veolia-has-plans-...,"['veolia', 'north', 'america', 'provid', 'envi..."
9586,"SunEdison: Too Big, Too Fast?",2023-11-08,NaN,Once the self-proclaimed “ leading renewable p...,solarindustrymag,http://www.solarindustrymag.com/online/issues/...,"['self', 'proclaim', 'lead', 'renew', 'power',..."


In [10]:
df_media.head(5)

,title,date,author,content,domain,url,token_content
0,Qatar to Slash Emissions as LNG Expansion Adva...,2021-01-13,NaN,Qatar Petroleum ( QP) is targeting aggressive ...,energyintel,https://www.energyintel.com/0000017b-a7dc-de4c...,"['qatar', 'petroleum', 'qp', 'target', 'aggres..."
1,India Launches Its First 700 MW PHWR,2021-01-15,NaN,Nuclear Power Corp. of India Ltd. ( NPCIL) syn...,energyintel,https://www.energyintel.com/0000017b-a7dc-de4c...,"['nuclear', 'power', 'corp', 'india', 'ltd', '..."
2,New Chapter for US-China Energy Trade,2021-01-20,NaN,New US President Joe Biden took office this we...,energyintel,https://www.energyintel.com/0000017b-a7dc-de4c...,"['new', 'us', 'presid', 'joe', 'biden', 'take'..."
3,Japan: Slow Restarts Cast Doubt on 2030 Energy...,2021-01-22,NaN,The slow pace of Japanese reactor restarts con...,energyintel,https://www.energyintel.com/0000017b-a7dc-de4c...,"['slow', 'pace', 'japanes', 'reactor', 'restar..."
4,NYC Pension Funds to Divest Fossil Fuel Shares,2021-01-25,NaN,Two of New York City's largest pension funds s...,energyintel,https://www.energyintel.com/0000017b-a7dc-de4c...,"['two', 'new', 'york', 'citi', 'larg', 'pensio..."


## patents

In [11]:

url = 'https://drive.google.com/file/d/1akyFU9QM35DlYni8_hv878Lxvi21cKX4/view?usp=sharing'
path = 'https://drive.google.com/uc?export=download&confirm=1&id='+url.split('/')[-2]
df_patents = pd.read_csv(path)

In [12]:
df_patents.head()

,publication_number,application_number,country_code,publication_date,inventor,title,abstract,abstract_lang,token_content
0,US-2022239235-A1,US-202217717397-A,US,2022-07-28,[],Adaptable DC-AC Inverter Drive System and Oper...,Disclosed is an adaptable DC-AC inverter syste...,['en'],"['disclos', 'adapt', 'dc', 'ac', 'invert', 'sy..."
1,US-2022239251-A1,US-202217580956-A,US,2022-07-28,[],System for providing the energy from a single ...,"In accordance with an example embodiment, a so...",['en'],"['accord', 'exampl', 'embodi', 'solar', 'energ..."
2,US-11396827-B2,US-202117606042-A,US,2022-07-26,[],Control method for optimizing solar-to-power e...,A control method for optimizing a solar-to-pow...,['en'],"['control', 'method', 'optim', 'solar', 'power..."
3,CN-114772674-A,CN-202210500131-A,CN,2022-07-22,[],Low-carbon running saline wastewater treatment...,The invention discloses a system and a method ...,"['zh', 'en']","['invent', 'disclos', 'system', 'method', 'tre..."
4,CN-217026795-U,CN-202220668705-U,CN,2022-07-22,[],Water ecological remediation device convenient...,The utility model discloses a water ecological...,"['zh', 'en']","['util', 'model', 'disclos', 'water', 'ecolog'..."


# Pre-processing

In the first try, no preprocessing applied. My guess is no preprocessing is needed as working with BERT normally does not require it.

## Testing KeyBert

Let's test with a tiny subset of the document(s) to see if things work as expected. Let's consider the following text from the first five patents:

In [13]:
test_string = ' '.join(df_patents.abstract.values[:5])
test_string

'Disclosed is an adaptable DC-AC inverter system and its operation. The system includes multiple DC input sources as input to provide a stable operation under various conditions. DC input sources may be added to the system or removed from the system without impacting the functionality of the system. The disclosed system is suited for solar energy harvesting in grid-connected or off-grid modes of operation. In accordance with an example embodiment, a solar energy system comprises: a solar energy structure comprising photovoltaic solar panels contiguously covering an area; a first inverter configured to receive power from a first string of photovoltaic solar panels, wherein the first inverter is configured to provide the power received at the first inverter to a first meter; and a second inverter configured to receive power from a second string of photovoltaic solar panels, wherein the second inverter is configured to provide the power received at the second inverter to a second meter. A

In [14]:
SumModelTest = Summarizer()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [19]:
import warnings
warnings.filterwarnings('ignore')

In [20]:
summary_test = SumModelTest(test_string, num_sentences=2)
print(summary_test)

Disclosed is an adaptable DC-AC inverter system and its operation. According to the utility model, the fixed sleeve and the lifting rod are arranged, the lifting rod is arranged in the fixed sleeve and positioned through the first bolt, the height of the lifting rod is adjusted by rotating the first bolt, the LED warning board is adopted as a warning device, and the solar panel is matched with the solar panel, absorbs solar energy in the daytime and converts the solar energy into electric energy to be stored in a power supply, so that the LED warning board has a strong warning effect at night.


## Extract from whole patent dataset

In [ ]:
patents = ' '.join(df_patents.title.values)
len(patents)

866508

In [ ]:
SumPatentModel = Summarizer()

In [ ]:
summary_patents = SumPatentModel(patents, num_sentences=25)
summary_patents

''

--> summarisation over all the text at once does not seem to work. And it is really a lot. So lets first summarise the documents on their own.

# Summarise single documents

Before going after the whole datasets, first summarize every single document to reduce the amount of characters drastically. Also do it in chunks to not risk memory overflows.

## Patents

In [21]:
SumPatentModel = Summarizer()

In [30]:
len(df_patents.abstract.values)

13199

In [39]:
BATCH_SIZE = 1000
patent_summaries = []
for batch in tqdm(chunked(df_patents.abstract.values, BATCH_SIZE)):
    batch_summaries = [SumPatentModel(abstract, num_sentences=2) for abstract in batch]
    patent_summaries.extend(batch_summaries)

14it [07:31, 32.28s/it]


In [40]:
len(patent_summaries)

13199

## Media articles

In media articles we have less documents, but way more text. Thats why we are cropping our text to 1500 chars. This is not ideal and we may miss some important ideas here or there - but it is the only way to run it with available compute time.

In [45]:
SumMediaModel = Summarizer()

In [43]:
len(df_media.content.values)

9588

In [ ]:
BATCH_SIZE = 500
media_summaries = []
for batch in tqdm(chunked(df_media.content.str[:1500].values, BATCH_SIZE)):
    batch_summaries = [SumMediaModel(content, num_sentences=2) for content in batch]
    media_summaries.extend(batch_summaries)

# Save to disk

Save the results to disk for later work

In [54]:
with open('media_summaries.txt', 'w') as file:
  for summary in media_summaries:
            file.write(f"{summary}\n")

In [55]:
with open('patent_summaries.txt', 'w') as file:
  for summary in patent_summaries:
            file.write(f"{summary}\n")

Author: Pascal Albisser